In [ ]:
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

Exploration des données

In [ ]:
df_plants = pd.read_csv("plants.csv")
df_plants.isnull().sum()

In [ ]:
df_plants.head()

In [ ]:
df_plants.shape

In [ ]:
df_plants.dtypes.value_counts()

In [ ]:
sns.set_context("notebook", font_scale=0.8, rc={"lines.linewidth": 2.0})
plt.figure(figsize=(10,10))
sns.pairplot(df_plants, hue='red_list_category', corner = True, palette = ['orange', 'blue'], height=4)
;

In [ ]:
sns.stripplot(x=df_plants['continent'],y=df_plants['group'],hue=df_plants['red_list_category'],dodge=True).set(title="Répartition du risque d'extinction par continent en fonction du type de plante") 
;

In [ ]:
sns.stripplot(x=df_plants['year_last_seen'],y=df_plants['group'],hue=df_plants['red_list_category'],dodge=True).set(title="Répartition du risque d'extinction par année de dernière observation en fonction du type de plante") 
plt.xticks(rotation=80)
;

In [ ]:
plt.figure(figsize=(25,8))
sns.stripplot(x=df_plants['country'],y=df_plants['group'],hue=df_plants['red_list_category']).set(title="Répartition du risque d'extinction par pays en fonction de l'espèce de plante")
plt.xticks(rotation=90)
;

In [ ]:
#for col in df_plants.select_dtypes('object'):
    #print(f'{col :#<50} {df_plants[col].unique()}{ df_plants[col].nunique() }')    #Fstring afin d'afficher valeurs prises par colonnes qualitatives

In [ ]:
df_plants.isnull().sum()

In [ ]:
plt.figure(figsize=(7,7))
sns.countplot(data=df_plants, x='continent', hue='red_list_category').set(title="Quantification du risque d'extinction par continent")
plt.xticks(rotation=60)
;

In [ ]:
plt.figure(figsize=(7,7))
sns.countplot(data=df_plants, x='group', hue='red_list_category').set(title="Quantification du risque d'extinction par espèce de plante")
plt.xticks(rotation=70)
;

In [ ]:
plt.figure(figsize=(7,7))
sns.countplot(data=df_plants, x='year_last_seen', hue='red_list_category').set(title="Quantification du risque d'extinction par année de dernière observation")
plt.xticks(rotation=70)
;

In [ ]:
plt.figure(figsize=(22,8))
sns.countplot(data=df_plants, x='country', hue='red_list_category',orient='v').set(title="Quantification du risque d'extinction par pays")
plt.xticks(rotation=90)
;

In [ ]:
df_plants.dtypes #types des variables

In [ ]:
plt.figure(figsize=(13,13))
sns.heatmap(df_plants.corr(numeric_only=True), square=True,annot=True,fmt=".2f").set(title="Corrélation des attributs quantitatifs")
;

In [ ]:
df_plants['red_list_category'].value_counts() #Le jeu de données est déséquilibré,la classe Extinct est                                        
                                             #plus fréquente que la classe Extinct in the Wild

In [ ]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(df_plants)
categorical_columns.remove('red_list_category')  #On n'encode pas red_list_category en OneHotEncoder mais en LabelEncoder
categorical_columns.remove('year_last_seen')    #On ne conserve pas cette variable du à ces valeurs nulles
 
df_categorical = df_plants[categorical_columns]
categorical_columns

In [ ]:
#Ici on encode toutes les variables catégorielles afin de les étudier dans le modèle
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
df_encoded = encoder.fit_transform(df_categorical)

df_target = df_plants['red_list_category']
df_plants = df_plants.drop('red_list_category',axis=1)

columns_encoded = encoder.get_feature_names_out(df_categorical.columns)
df_encoded = pd.DataFrame(df_encoded, columns=columns_encoded)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_target = le.fit_transform(df_target)
df_target = pd.DataFrame(df_target, columns=['red_list_category'])


In [ ]:
from sklearn import preprocessing

df = pd.concat([df_encoded,df_plants,df_target], axis=1)

df = df.drop(['binomial_name','country','continent','group'], axis=1) #on peut se séparer des variables catégorielles non encodées du début
df = df.drop('year_last_seen', axis=1)                                #et de cette variable avec des valeurs nulles

df

In [ ]:
print('#######Avant conversion des types')
df.dtypes

In [ ]:
import numpy as np
nom_col = list(df.columns)
nom_col.remove('red_list_category')

#print(nom_col)
for col in nom_col:
    df[col] = df[col].astype(np.int64)  #chaque colonne est convertie en int64 (certains etaient en int32)
print('#######Après conversion des types')
df.dtypes

In [ ]:
df.shape

Rééchantillonage du dataset pour pallier au problème de déséquilibre

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
#On utilise la technique d'oversampling ici
#On augmente la classe minoritaire de Extinct in the wild afin d'avoir des échantillons équilibrés
from imblearn.over_sampling import RandomOverSampler

X = df[nom_col]
y = df[['red_list_category']]

print('X avant oversampling: ',X.shape)
print('y avant oversampling: ',y.shape)
print('-------------\n')

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)

from collections import Counter
print('X après oversampling: ', X_resampled.shape)
print('y après oversampling: ', y_resampled.shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled , test_size = 0.2,random_state=0)

print('Jeu d entrainement du modèle:', X_train.shape)
print('Jeu de test:', X_test.shape)

In [ ]:
scaler = StandardScaler()    #on normalise
scaler.fit_transform(X_train)
scaler.fit_transform(X_test)

Choix du modèle Knn - estimation de la performance

In [ ]:
from sklearn.model_selection import validation_curve

In [ ]:
list_score_classifiers = []  #Liste regroupe les scores de chaque méthodes qu'on étudie

In [ ]:
model_v_knn = KNeighborsClassifier()
#Ce sont les différentes valeurs à tester pour k
k = np.arange(1,100)

train_score , validation_score = validation_curve(model_v_knn, X_train, y_train.values.ravel(),
                                                  param_name='n_neighbors', param_range=k , cv=5, 
                                                  scoring = "accuracy" )                           #cv est le nombre de découpes pour la cross-validation
print(validation_score) # ce qui est en bas représente les score de validation sur chaque découpe (cf cross-validation) pour un voisin donné

#On représente le score obtenu sur la validation et l'apprentissage
plt.plot(k, validation_score.mean(axis=1), label='validation') #on fait la moyenne des score de validation pour avoir le score validation
plt.plot(k, train_score.mean(axis=1), label='train')

plt.ylabel('score')
plt.xlabel('n_neighbors')
plt.legend()
plt.title('Accuracy pour la méthode K plus proches voisins')
plt.show()

##il semblerait que le nombre de voisin k optimal ici  est k = 2  avec 0.946813978 d'accuracy
#k = 1  -> 0.943946558
#k = 2  -> 0.946813978  -> meilleur
#k = 3  -> 0.9252517960000001
#k = 4  -> 0.9209352519999999
#k = 5  -> 0.897923946

#cela prend 25secondes en moyenne à l'execution

In [ ]:
#Ceci est le score d'accuracy de la cross-validation pour k=2, cf cellule du haut avec les scores
#On va chercher à optimiser ce score , dans la cellule plus bas par la sélection sur Grille (GridSearch)
print(np.array([0.96428571, 0.96402878, 0.94964029, 0.95683453, 0.89928058]).mean())    #le k trouvé

Suite choix du modèle : Optimisation de notre modèle, utilisation de GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid= {'n_neighbors': [2, 20, 30, 50, 70, 90, 100],
             'metric':['euclidean','manhattan','hamming'],
             'weights': ['uniform', 'distance']
            }

grid_knn = GridSearchCV(KNeighborsClassifier(algorithm='auto'),param_grid,cv=5,scoring='accuracy')

grid_knn.fit(X_train, y_train.values.ravel())     #9 secondes environ

In [ ]:
#Le modèle qui a obtenu le meilleur score
#On a un score d'accuracy    de 0.9583247687564235  > 0.946813978
grid_knn.best_score_

In [ ]:
#Les meilleurs paramètres trouvés par GridSearchCV pour le modèle de K plus proches voisins
grid_knn.best_params_

In [ ]:
#On sauvegarde le meilleur estimateur calculé par GridSearchCV
knn_model = grid_knn.best_estimator_

Etape d'estimation des performances finales du modèle Knn choisi (erreur prédiction, erreur généralisation sur nouvelle donnée)

In [ ]:
list_score_classifiers.append(knn_model.score(X_test,y_test.values.ravel())) #ajout dans la liste des scores des classifieurs étudiés

#Ici on teste le modèle sur les jeux de test
#C'est l'accuracy sur les données de test pour estimer la performance de notre modèle K plus proches voisins
knn_model.score(X_test,y_test.values.ravel())

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
#Evaluation du modèle K plus proches voisins par matrice de confusion
cm = confusion_matrix(y_test.values.ravel(),knn_model.predict(X_test))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Extinct', 'Extinct in the Wild'])
disp.plot()
disp.ax_.set_title("Matrice de confusion modèle K plus proches voisins")
;

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print('Métriques de performance pour K plus proches voisins\n\n', 
    classification_report(y_test.values.ravel(), knn_model.predict(X_test), target_names=['Extinct','Extinct in the Wild']))

Forêt aléatoire

In [ ]:
from sklearn.ensemble import RandomForestClassifier
param_grid = { 
    'n_estimators': [5, 20, 50], #nombre darbre pour une foret
    'criterion' :['gini', 'entropy'],
      'max_depth' : [2,4,6,8]  #profondeur arbre
    
}
grid_rfc = GridSearchCV(
    RandomForestClassifier(), 
    param_grid,     
    cv=5,           # nombre découpes de validation croisée
    scoring='accuracy'   # score à optimiser
)

In [ ]:
grid_rfc.fit(X_train, y_train.values.ravel())   #6 secondes execution en moyenne

In [ ]:
#Les meilleurs paramètres trouvés par GridSearch pour le modèle de forêt aléatoire
grid_rfc.best_params_

In [ ]:
#On sauvegarde le meilleur estimateur calculé par GridSearchCV
rfc_model = grid_rfc.best_estimator_

Etape d'estimation des performances finales du modèle Forêt aléatoire (généralisation sur nouvelle donnée)

In [ ]:
list_score_classifiers.append(rfc_model.score(X_test,y_test.values.ravel())) #ajout dans la liste des scores des classifieurs étudiés

#Ici on teste le modèle sur les jeux de test
#C'est l'accuracy sur les données de test pour estimer la performance de notre modèle Forêt aléatoire
rfc_model.score(X_test,y_test.values.ravel())

In [ ]:
#Evaluation du modèle Forêt aléatoire par matrice de confusion
cm = confusion_matrix(y_test.values.ravel(),rfc_model.predict(X_test))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Extinct','Extinct in the Wild'])
disp.plot()
disp.ax_.set_title("Matrice de confusion modèle Forêt aléatoire")
;

In [ ]:
print('Métriques de performance pour Forêt aléatoire\n\n',
      classification_report(y_test.values.ravel(), rfc_model.predict(X_test), target_names=['Extinct','Extinct in the Wild']))

Arbre de décision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
param_grid = { 
     'max_leaf_nodes': [2 , 20 , 30, 50],
     'min_samples_split': [2, 3, 4],
      'max_depth' : [2,4,6] , #profondeur arbre
       'criterion' :['gini', 'entropy']  #criteres aux noeuds
}
grid_dtc = GridSearchCV(
    DecisionTreeClassifier(), 
    param_grid,     
    cv=5,           #5 découpes de validation croisée
    scoring='accuracy'   # score à optimiser
)

In [ ]:
grid_dtc.fit(X_train, y_train.values.ravel())  #7 secondes execution en moyenne

In [ ]:
#Les meilleurs paramètres trouvés par GridSearch pour le modèle d'arbre de décision
grid_dtc.best_params_

In [ ]:
#On sauvegarde le meilleur estimateur calculé par GridSearchCV
dtc_model = grid_dtc.best_estimator_

Etape d'estimation des performances finales du modèle Abre de décision (généralisation sur nouvelle donnée)

In [ ]:
list_score_classifiers.append(dtc_model.score(X_test,y_test.values.ravel())) #ajout dans la liste des scores des classifieurs étudiés

#Ici on teste le modèle sur les jeux de test
#C'est l'accuracy sur les données de test pour estimer la performance de notre modèle Arbre de décision
dtc_model.score(X_test,y_test.values.ravel())

In [ ]:
#Evaluation du modèle Arbre de décision par matrice de confusion
cm = confusion_matrix(y_test.values.ravel(),dtc_model.predict(X_test))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Extinct','Extinct in the Wild'])
disp.plot()
disp.ax_.set_title("Matrice de confusion modèle Arbre de décision")
;

In [ ]:
print('Métriques de performance pour Arbre de décision\n\n', 
    classification_report(y_test.values.ravel(), dtc_model.predict(X_test), target_names=['Extinct','Extinct in the Wild']))

XGBoost


In [ ]:
from xgboost import XGBClassifier

In [ ]:
parameters = {
    'booster': ['gbtree','gblinear'],
    'n_estimators': [5,20,50],  #nombre darbre pour une foret
    'learning_rate': [0.1, 0.01, 0.05] #controler le taux apprentissage
}

grid_xgboost = GridSearchCV(
    estimator=XGBClassifier(),
    param_grid=parameters,
    scoring = 'accuracy',     #score à optimiser
    cv = 5     #5 découpes de validation croisée
)

In [ ]:
grid_xgboost.fit(X_train, y_train.values.ravel())    #execution 31 secondes en moyenne

In [ ]:
#Les meilleurs paramètres trouvés par GridSearch pour le modèle XGboost
grid_xgboost.best_params_

In [ ]:
#On sauvegarde le meilleur estimateur calculé par GridSearchCV
xgbc_model = grid_xgboost.best_estimator_

Etape d'estimation des performances finales du modèle XGBoost choisi (erreur prédiction, erreur généralisation sur nouvelle donnée)


In [ ]:
list_score_classifiers.append(xgbc_model.score(X_test,y_test.values.ravel())) #ajout dans la liste des scores des classifieurs étudiés

#Ici on teste le modèle sur les jeux de test
#C'est l'accuracy sur les données de test pour estimer la performance de notre modèle XGboost
xgbc_model.score(X_test,y_test.values.ravel())

In [ ]:
#Evaluation du modèle XGboost par matrice de confusion
cm = confusion_matrix(y_test.values.ravel(),xgbc_model.predict(X_test))

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Extinct','Extinct in the Wild'])
disp.plot()
disp.ax_.set_title("Matrice de confusion modèle XGboost")
;

In [ ]:
print('Métriques de performance pour XGboost\n\n',
    classification_report(y_test.values.ravel(), xgbc_model.predict(X_test), target_names=['Extinct','Extinct in the Wild']))

Résumé des performances en terme d'accuracy

In [ ]:
list_score_classifiers = [round(item, 3) for item in list_score_classifiers]
list_score_classifiers.sort()#Trier les scores
list_score_classifiers

In [ ]:
axis_x = ['Arbre de décision','Forêt aléatoire','XGBoost','K plus proches voisins']

plt.figure(figsize=(12, 5))
plt.barh(axis_x,list_score_classifiers, color=['red', 'green', 'grey', 'brown'])
plt.xlabel("Accuracy")
plt.ylabel("Méthodes de classification")
plt.title("Comparaison des méthodes de classification")
for index, value in enumerate(list_score_classifiers):
    plt.text(value, index,
             str(value))